# SBUS Demo

> Source data: https://ichoosr.egnyte.com/navigate/folder/701cce7f-a797-47c6-bcae-7ee2e3242e8c
> 
> Confluence: https://ichoosr.atlassian.net/wiki/x/IYBwMQE?atlOrigin=eyJpIjoiMzg2YWQxZmI2NjUyNDg1Y2E2YjMyZTViOGRjOWFmOGEiLCJwIjoiYyJ9

In [ ]:
%reload_ext setup

## Demo

In [ ]:
# from igent.workflows.p1m1m2c import run_workflow

# dir = "../data/final"
# line = "sbus"
# await run_workflow(
#     model="openai", # "openai" or "azure" or "vm_deepseek"
#     stream=True,
#     business_line=line,
#     registrations_file=f"{dir}/{line}_mock_registrations_10.json",
#     offers_file=f"{dir}/{line}_mock_offers.json",
#     matches_file=f"{dir}/matches.json",
#     pos_file=f"{dir}/pos.json",
#     max_items=10,
#     stats_file=f"{dir}/execution_times.csv"
# )

## Final

In [ ]:
from igent.workflows.p1m1m2c import run_workflow

line = "sbus"
dir = "../data/final"
sufix = "_50pct_nobattery_100"
await run_workflow(
    model="openai", # "openai" or "azure" or "vm_deepseek"
    stream=True,
    business_line=line,
    registrations_file=f"{dir}/{line}_mock_registrations{sufix}.json",
    offers_file=f"{dir}/{line}_mock_offers{sufix}.json",
    matches_file=f"{dir}/matches{sufix}.json",
    pos_file=f"{dir}/pos{sufix}.json",
    max_items=100,
    stats_file=f"{dir}/execution_times{sufix}.csv"
)